# Setup

In [ ]:

# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install fastai --upgrade -q
from fastai.vision.all import *
# from fastai.callback.cutmix import *

!pip install wwf -q
!pip install timm -q
from wwf.vision.timm import *

# !pip install efficientnet_pytorch -q
# from efficientnet_pytorch import EfficientNet

# Import data

In [ ]:
path = Path('../input/planets-dataset/planet/planet')

In [ ]:
train_df = pd.read_csv(path/'train_classes.csv')
train_df

In [ ]:
def get_x(r):
    return path/'train-jpg'/(r['image_name']+'.jpg')

def get_y(r):
    return r['tags'].split()

def get_data(size=224,bs=64,data_df=train_df):
    dblock = DataBlock(blocks=(ImageBlock, MultiCategoryBlock),
                       splitter=RandomSplitter(seed=42),
                       get_x=get_x, 
                       get_y=get_y,
                       item_tfms = Resize(size),
                       batch_tfms = [*aug_transforms(flip_vert=True,max_warp=0),
                                     Normalize.from_stats(*imagenet_stats)]
                      )
    return dblock.dataloaders(data_df,bs=bs)

In [ ]:
dls = get_data(300,40)

In [ ]:
dls.show_batch(nrows=1, ncols=3)

# Training

In [ ]:

f2samples = FBetaMulti(beta=2,average='samples',thresh=0.2)

# model = EfficientNet.from_pretrained('efficientnet-b7', get_c(dls))
metrics = [partial(accuracy_multi, thresh=0.2), f2samples]
cbs = [MixUp]

In [ ]:
learn = timm_learner(dls, 'efficientnet_b3', metrics=metrics, cbs=cbs)

In [ ]:
import gc
gc.collect

In [ ]:
learn.fine_tune(12, base_lr=3e-2, freeze_epochs=4)

****

In [ ]:
file_path = Path('../input/planets-dataset/test-jpg-additional/test-jpg-additional')
test_path = Path('../input/planets-dataset/planet/planet/test-jpg')
submission_df = pd.read_csv(path/'sample_submission.csv')
testing_path = (submission_df['image_name'] + '.jpg').apply(lambda x: test_path/x if x.startswith('test') else file_path/x)

def prediction(filename='submission.csv', tta=False):
    tst_dl = learn.dls.test_dl(testing_path)
    if tta:
        predictions = learn.tta(dl = tst_dl)
    else:
        predictions = learn.get_preds(dl = tst_dl)
    predlist = [' '.join(learn.dls.vocab[i]) for i in (predictions[0] > 0.2)]

    df = submission_df
    df['tags'] = predlist

    df.to_csv(filename, index=False)
    return df

In [ ]:
prediction('submission_tta.csv', tta=True)